Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [x] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [x] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

### Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [3]:
df.shape

(48817, 34)

In [4]:
df.head(2).T

,0,1
bathrooms,1.5,1
bedrooms,3,2
created,2016-06-24 07:54:24,2016-06-12 12:19:27
description,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,
display_address,Metropolitan Avenue,Columbus Avenue
latitude,40.7145,40.7947
longitude,-73.9425,-73.9667
price,3000,5465
street_address,792 Metropolitan Avenue,808 Columbus Avenue
interest_level,medium,low


In [5]:
df['created'] = pd.to_datetime(df['created'])

In [6]:
# https://stackoverflow.com/questions/22898824/filtering-pandas-dataframes-on-dates
# Filter data by year

# df.query('20130101 < date < 20130201')

train = df.query('20160401 < created < 20160531')
test  = df.query('20160601 < created < 20160701')

train.shape, test.shape

((31839, 34), (16973, 34))

In [7]:
test['created'].head()

0    2016-06-24 07:54:24
1    2016-06-12 12:19:27
11   2016-06-03 03:21:22
14   2016-06-01 03:11:01
24   2016-06-07 04:39:56
Name: created, dtype: datetime64[ns]

In [48]:
net = train.shape[0] + test.shape[0]
difference = df.shape[0] - net

print(f'{difference} missing values')

5 missing values


### Engineer at least two new features.

In [9]:
df2 = df

In [10]:
df.shape[1], df2.shape[1]

(34, 34)

In [11]:
df2['over_2000'] = df2['price'] > 2000

In [12]:
df2.over_2000.value_counts()

True     43756
False     5061
Name: over_2000, dtype: int64

In [13]:
df2['low_interest'] = df2.interest_level == 'low'

In [14]:
df2.low_interest.value_counts()

True     33946
False    14871
Name: low_interest, dtype: int64

In [15]:
df.shape[1], df2.shape[1]

(36, 36)

In [16]:
# I don't understand why my new features are being 
# added to both df2 and df

### Fit a linear regression model with at least two features.

In [18]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [23]:
# Baseline

train['price'].mean()

3575.737743019567

In [35]:
features = [
            'bathrooms',
            'bedrooms',
            'longitude',
            'latitude',
           ]

target = 'price'

X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Get the model's coefficients and intercept.

In [31]:
model.coef_

# features = [
#             'bathrooms',
#             'bedrooms',
#             'longitude',
#             'latitude',
#            ]

array([  2002.52555996,    428.59081066, -16327.9646863 ,   1863.66270886])

In [32]:
model.intercept_

# target = 'price'

-1283261.8184454648

### Get regression metrics RMSE, MAE, and  𝑅2 , for both the train and test data.

In [42]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [44]:
# https://stackoverflow.com/questions/17197492/is-there-a-library-function-for-root-mean-square-error-rmse-in-python#18623635

from sklearn.metrics import mean_squared_error
from math import sqrt

# rmse = sqrt(mean_squared_error(y_actual, y_predicted))

rmse_train = sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = sqrt(mean_squared_error(y_test, y_pred_test))

rmse_train, rmse_test

(1146.220016647726, 1131.196181842376)

In [36]:
from sklearn.metrics import mean_absolute_error

mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

mae_train, mae_test

(739.2051470439327, 744.991379197823)

In [47]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html

from sklearn.metrics import r2_score

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

r2_train, r2_test

(0.5769196300014992, 0.5882895606120184)

### Stretch Goal - If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression

In [ ]:
# 59 - 82

* An interaction effect is the result of two features yielding
a higher target outcome than either one individually
https://chrisalbon.com/machine_learning/linear_regression/adding_interaction_terms/

* The t-distribution is approximately equal to a normal distribution when n > 30

* A small p-value suggests it's unlikely to observe a substantial association between the feature and target due to chance without any association between the two (5% or 1% are the typical standards for rejecting the null hypothesis)

* Residual Standard Error (RSE) and R^2 are typically used to define the quality of a linear regression model

* RSE is the average amount that the target will deviate from the true regression line. A large RSE indicates a poor predictive model

* Residual Sum of Squares (RSS) measures the amount of variability that is left unexplained after performing regression

* R^2 measures the proportion of variability in the target that can be explained through the features

* An F-statistic close to 1 indicates no relationship between features and a target

* If n is large, an F-statistic slightly larger than 1 might justify rejecting the null hypothesis. Conversely, if n is small then a large F-statistic is required to do the same

* The F-statistic is the preferred test when the number of features is small relative to sample size n

* When features > n, forward selection should be used. This is the process of adding features to a model one by one until a criteria is satisfied

* Backward selection involes including all features in a model and removing each of them 1 by 1 in order of highest p value until we satisfy a defined criteria. This can't be used if p > n (where p is our features) 